In [1]:
import sys
import random
import json
from time import time, sleep
from PyQt5 import QtGui, QtWidgets, QtCore, uic
from PyQt5.QtCore import QPropertyAnimation, QPoint, QEasingCurve

In [3]:
class HomeWindow(QtWidgets.QMainWindow):
    def __init__(self, *args, **kwargs):
        super(HomeWindow, self).__init__(*args, **kwargs)

        #load UI created using the GUI Tool
        self.ui = uic.loadUi('reactionTime.ui', self)
        
        #connect buttons to their actions
        self.startButton.clicked.connect(self.pressedButton)
        self.clearButton.clicked.connect(self.clearData)
        self.saveButton.clicked.connect(self.saveFile)
        
        #create the animation and define its properties
        self.animation = QPropertyAnimation(self.fallingBall, b'pos')
        self.animation.setStartValue(QPoint(120,-50))
        self.animation.setEndValue(QPoint(120, 500))
        self.animation.setDuration(2000)
        self.animation.setLoopCount(1)
        self.animation.valueChanged.connect(self.ballPosition)
        self.animation.finished.connect(self.animationFinished)
        self.fallingBall.move(120,-50)
        self.crossingBar.move(120,-50)
        
        #define the position of the bar
        self.PositionBar = 300
        
        #variable that will carry the reaction time 
        self.startTime = 0
        
        #add items to convo box
        self.velocityProfileSelector.addItem("Linear")
        self.velocityProfileSelector.addItem("InOutQuad")
        self.velocityProfileSelector.addItem("InQuad")
        self.velocityProfileSelector.addItem("OutQuad")
        self.velocityProfileSelector.addItem("OutInQuad")
        self.velocityProfileSelector.addItem("InOutCubic")
        self.velocityProfileSelector.addItem("InCubic")
        self.velocityProfileSelector.addItem("OutCubic")
        self.velocityProfileSelector.addItem("OutInCubic")

        #empty data dictionary
        self.data = {'time':[],
                     'velocity':[],
                     'profile':[],
                     'random': []
                    }

    #function to clear the table showing the data    
    def clearData(self): 
        #this only works if the timer is not running
        if (self.startTime == 0):
            self.data = {'time':[],
                         'velocity':[],
                         'profile':[],
                         'random': []
                        }
            self.fillTable()
        
    #function to fill the table usinf the variable self.data     
    def fillTable(self):
        # fillTable
        self.tableWidget.setRowCount(len(self.data['time']))
        self.tableWidget.setColumnCount(4)
        for n, key in enumerate(self.data.keys()):
            for m, item in enumerate(self.data[key]):
                newitem = QtWidgets.QTableWidgetItem(item)
                self.tableWidget.setItem(m, n, newitem)
                
        self.tableWidget.setHorizontalHeaderLabels(['Reaction (ms)','Fall Time (ms)', 'Profile', 'Random'])

    #action when the main button is pressed
    def pressedButton(self):
        #get the desired velocity from the slider
        velocity = self.velocitySlider.value()
        #get the desired velocity profile from the selector
        curve = self.velocityProfileSelector.currentText()
        
        #in this case, the user pressed 'Start'
        if (self.startButton.text() == 'Start'):
            #change the text in the button
            self.startButton.setText('Cross')
            
            #set the animation velocity and profile based on user input
            self.animation.setDuration(int(2000*(velocity/100)))
            setCurve = "self.animation.setEasingCurve(QEasingCurve."+curve+")"
            eval(setCurve)
                    
            #modify the position on the bar randomly if that option is selected
            if self.randomSelector.isChecked() : #update bar position every pass?
                self.PositionBar = random.randint(100, 400) #generate a random number between 100 and 400
                self.crossingBar.move(0,self.PositionBar)
            else:
                self.crossingBar.move(0,self.PositionBar)
                
            #start the animation
            self.animation.start()
            
        else: #in this case, the user pressed 'Cross'
            if (self.startTime !=0): #verify that the timer is active 
                #store data in the data dictionary
                self.data['time'].append(str((time()-self.startTime)*1000))
                self.data['velocity'].append(str(int(2000*(velocity/100))))
                self.data['profile'].append(curve)
                self.data['random'].append(str(self.randomSelector.isChecked()))
                
                #reset the start time 
                self.startTime=0
                #change the text in the button
                self.startButton.setText('Start')
                #add the data to the table
                self.fillTable()

    #this function will be called everytime the animation is updated
    def ballPosition(self):
        #did the ball crossed the bar
        #self.fallingBall.pos().y() -> provides the y() position of the fallingBall (the center)
        if (self.fallingBall.pos().y()+25 >= self.PositionBar):
            #start the timer once the ball crosses the line
            self.startTime = time()
        else:
            self.startTime = 0 
    
    #this function will be called once the animation is finished
    def animationFinished(self):
        #animation is done, reset everything
        sleep(0.5) #wait 500ms before moving forward
        self.startButton.setText('Start')
        self.fallingBall.move(120,-50)
        self.crossingBar.move(120,-50)
        self.startTime=0
        
    def saveFile(self):
        name = QtWidgets.QFileDialog.getSaveFileName(self, 'Save File')
        fileName = name[0]+'.json'
        with open(fileName, "w") as outfile:
            json.dump(self.data, outfile)
             
        
#run the pyqt code        
def main():
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtWidgets.QApplication(sys.argv)

    w = HomeWindow()
    w.setWindowTitle('Reaction Time')
    w.show()

    sys.exit(app.exec_())


if __name__ == '__main__':
    main()

SystemExit: 0